Arina Yartseva BS21-AAI-01

a.yartseva@innopolis.university 

# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

You may also want to implement:
- spell-checking for a concrete language - Russian, Tatar, etc. - any one you know, such that the solution accounts for language specifics,
- some recent (or not very recent) paper on this topic,
- solution which takes into account keyboard layout and associated misspellings,
- efficiency improvement to make the solution faster,
- any other idea of yours to improve the Norvig’s solution.

IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

In [1]:
import numpy as np
from nltk.tokenize import word_tokenize, sent_tokenize
import re
from collections import Counter
from functools import cache
import pandas as pd
from tqdm import tqdm
import random
import difflib

In [2]:
# Set random seed for reproducibility
random.seed(21)
np.random.seed(21)

In [3]:
words_lambda = lambda text: re.findall(r'\w+', text.lower())

WORDS = Counter(words_lambda(open('big.txt').read()))

In [4]:
class NGram:
    def __init__(self, filename: str):
        """
        Initialize the n-gram model

        :param filename: the filename of the n-gram model
        """
        self.df = pd.read_csv(filename, header=None, delimiter='\t', encoding='latin-1')
        self.voco = set(self.df[1]).union(set(self.df[2]))
        self.sum_all = self.df[0].sum()

    @property
    def n(self):
        """
        Get the n of the n-gram

        :return: the n of the n-gram
        """
        # The n-gram order is determined by the number of word columns in the dataframe.
        return len(self.df.columns) - 1

    @cache
    def get_prob(self, context: str, word: str) -> float:
        """
        Get the probability of a word given a context

        :param context: the context
        :param word: the word
        :return: the probability of the word given the context
        """
        # Filter the dataframe for rows where the context matches.
        context_df = self.df[self.df[1] == context]

        # If the context is not found, return a uniform probability.
        if not len(context_df):
            return 1.0 / self.sum_all
        
        # Filter for the specific word within the matching context.
        word_context = context_df[context_df.iloc[:, -1] == word]
        word_context_s = 1.0

        # If the word is found within the context, adjust the score based on frequency.
        if len(word_context):
            word_context_s += float(word_context.iloc[0, 0])

        # Return the normalized probability.
        return word_context_s / self.sum_all

    def get_sentence_probability(self, sentence: list[str]) -> float:
        """
        Get the probability of a sentence

        :param sentence: the sentence
        :return: the probability of the sentence
        """
        result = 1.0

        # Iterate through the sentence to calculate probabilities for each n-gram within it.
        for i in range(len(sentence) - self.n + 1):
            a, b = sentence[i: i + self.n]
            r = self.get_prob(a, b)
            result *= r
            
        # Return the cumulative probability of the sentence.
        return result
    

n2g = NGram('w2_.txt')

In [5]:
unique_vocab = set(WORDS.keys())
vocob = n2g.voco.union(unique_vocab)

In [6]:
E1_coef = 0.9999999
E2_coef = 0.0000001
Ecorr_coef = 700000.0

In [7]:
def known(words: set[str]) -> set[str]:
    """
    The subset of `words` that appear in the dictionary of WORDS.
    
    :param words: the words
    :return: the subset of `words` that appear in the dictionary of WORDS
    """
    return set(w for w in words if w in WORDS)

In [8]:
def calc_unigram_prob_in_edits(edits: set[str], coef: float) -> dict[str, float]:
    """
    Calculate the probability of a word given the edits

    :param edits: the edits
    :param coef: the coefficient
    :return: the probability of a word given the edits
    """
    known_words = []
    known_probs = []

    # Loop through each edit and, if it's known, calculate its probability.
    for edit in known(edits):
        known_words.append(edit)
        known_probs.append(WORDS.get(edit, 0))
    
    # Convert the list of probabilities to a numpy array for vectorized operations.
    known_probs = np.array(known_probs)
    
    # Apply Laplace smoothing (adding 1 to each count) and multiply by the given coefficient.
    # Laplace smoothing is used to handle zero probabilities for unseen words.
    known_probs = (known_probs + 1) * coef
    
    # Return a dictionary mapping each known word (edit) to its adjusted probability.
    return {word: prob for word, prob in zip(known_words, known_probs)}

In [9]:
def norm_values(d : dict[str, float]) -> dict[str, float]:
    """
    Normalize the values of a dictionary

    :param d: the dictionary
    :return: the normalized dictionary
    """
    total = sum(d.values())
    for k in d:
        d[k] = d[k] / total
    return d

In [10]:
@cache
def edits1(word: str) -> set[str]:
    """
    All edits that are one edit away from `word`.

    :param word: the word
    :return: all edits that are one edit away from `word`
    """
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word: str) -> set[str]:
    """
    All edits that are two edits away from `word`.
    
    :param word: the word
    :return: all edits that are two edits away from `word`
    """
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [11]:
def candidates(word: str):
    """
    Generate possible spelling corrections for word.

    :param word: the word
    :return: the possible spelling corrections for word
    """
    if len(word) == 1:
        return Counter({word: 1.0})

    known_probs = {}

    # Generate and update probabilities for corrections that are two edits away (for words longer than 3 characters).
    if len(word) > 3:
        known_probs.update(calc_unigram_prob_in_edits(edits2(word), E2_coef))

    # Generate and update probabilities for corrections that are one edit away.
    known_probs.update(calc_unigram_prob_in_edits(edits1(word), E1_coef))
    
    # If the original word is known, assign it a high probability.
    if word in WORDS:
        known_probs.update(calc_unigram_prob_in_edits({word}, Ecorr_coef))

    # If no known corrections are found, return the original word as the only correction.
    if not known_probs:
        return Counter({word: 1.0})

    # Normalize the probabilities of the corrections and return them.
    return Counter(norm_values(known_probs))

In [12]:
BeamSize = 4

def correct_sentence(sentence: str) -> str:
    """
    Correct a sentence

    :param sentence: the sentence
    :return: the corrected sentence
    """
    # Tokenize the input sentence and convert it to lowercase
    tokenized = word_tokenize(sentence.lower())
    
    # Initialize candidates for the first word
    current_candidates = candidates(tokenized[0])
    current_candidates = current_candidates.most_common(BeamSize)
    
    # Iterate over the rest of the words in the sentence
    for i in tqdm(range(1, len(tokenized)), desc="Correcting sentence"):
        # Generate candidates for the next word
        next_candidates = candidates(tokenized[i])
        next_candidates = next_candidates.most_common(BeamSize)
        
        # Prepare to collect new candidate combinations
        new_candidates = {}
        
        # Combine each pair of current and next candidates
        for first_word, first_prob in current_candidates:
            last_word_of_first = first_word.split(' ')[-1]
            for second_word, second_prob in next_candidates:
                # Calculate the combined probability of the two-word sequence
                ng_prob = n2g.get_prob(last_word_of_first, second_word)
                combined_prob = first_prob * ng_prob * second_prob
                new_sequence = first_word + ' ' + second_word
                new_candidates[new_sequence] = combined_prob
        
        # Select the top BS candidates based on their combined probabilities
        current_candidates = Counter(new_candidates).most_common(BeamSize)
    
    # Return the most probable corrected sentence
    return current_candidates[0][0]

## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

In this implementation, several decisions were made regarding the n-gram dataset, weights for edit probabilities, and beam search parameters. Here, we give the justifications of these decisions.

N-gram dataset: The model used in this implementation treats words as occurring in pairs. Therefore, this dataset was chosen because it allows for a reasonable compromise between context capture and computation efficiency. Our 5-gram dataset is too small to be of use for this model.

Edit probabilities weights: The respective weights given to the edit probabilities are E1_coef = 0.9999999 and E2_coef = 0.0000001. This is on the general assumption that single edit (edit1) corrections are much more probable than double-edit (edit2) ones. This is the reasonable assumption, since in most cases, spelling mistakes are more likely simple typographical errors or single-character errors. Ecorr_coef is 700000.0 in order to give a far higher probability to the known original word, should it be known. Then, it is the probable correct spelling for the original word. The coefficients were empirically chosen by the test set.

Search parameters (use beam search): search procedure attempts to find the most likely sequences of corrections for the given sentence. The beam size was taken as equal to 4. The beam size was set empirically, using a test set.

## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity. Compare your solution to the Norvig's corrector, and report the accuracies.

### Norvig corrector
https://norvig.com/spell-correct.html

In [13]:
import re
from collections import Counter

def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open('big.txt').read()))

def P(word, N=sum(WORDS.values())):
    "Probability of `word`."
    return WORDS[word] / N

def correction(word):
    "Most probable spelling correction for word."
    return max(n_candidates(word), key=P)

def n_candidates(word):
    "Generate possible spelling corrections for word."
    return (n_known([word]) or n_known(n_edits1(word)) or n_known(n_edits2(word)) or [word])

def n_known(words):
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def n_edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def n_edits2(word):
    "All edits that are two edits away from `word`."
    return (e2 for e1 in n_edits1(word) for e2 in n_edits1(e1))

def correct_sentence_norvig(sentence):
    tokeinzed = word_tokenize(sentence.lower())
    corr = [correction(e) for e in tokeinzed]
    return corr

### Test set generation

In [14]:
text = "Holmes had sat up upon the couch, and I saw him motion like a man who is in need of air. A maid rushed across and threw open the window. At the same instant I saw him raise his hand and at the signal I tossed my rocket into the room with a cry of Fire! The word was no sooner out of my mouth than the whole crowd of spectators, well dressed and ill--gentlemen, ostlers, and servant maids--joined in a general shriek of Fire! Thick clouds of smoke curled through the room and out at the open window. I caught a glimpse of rushing figures, and a moment later the voice of Holmes from within assuring them that it was a false alarm. Slipping through the shouting crowd I made my way to the corner of the street, and in ten minutes was rejoiced to find my friend's arm in mine, and to get away from the scene of uproar. He walked swiftly and in silence for some few minutes until we had turned down one of the quiet streets which lead towards the Edgeware Road."

sentences = sent_tokenize(text)
sentences = [re.sub(r'[^a-zA-Z ]', '', e).lower() for e in sentences]

In [15]:
def deletion(L: str, R: str, c: str) -> str:
    """
    Perform deletion edit: Remove the first character from R.
    
    :param L: the left part of the string
    :param R: the right part of the string
    :param c: the character
    :return: the string after deletion
    """
    return L + R[1:] if R and R[0] != ' ' else L + R

def transposition(L: str, R: str, c: str) -> str:
    """
    Perform transposition edit: Swap the first two characters in R.
    
    :param L: the left part of the string
    :param R: the right part of the string
    :param c: the character
    :return the string after transposition
    """
    return L + R[1] + R[0] + R[2:] if len(R) > 1 else L + R

def replacement(L: str, R: str, c: str) -> str:
    """
    Perform replacement edit: Replace the first character in R with c.
    
    :param L: the left part of the string
    :param R: the right part of the string
    :param c: the character to replace
    """
    return L + c + R[1:] if R else L + c

def insertion(L: str, R: str, c: str) -> str:
    """
    Perform insertion edit: Insert c before R.
    
    :param L: the left part of the string
    :param R: the right part of the string
    :param c: the character to insert
    """
    return L + c + R

# List of operation functions
ops = [deletion, transposition, replacement, insertion]

letters = 'abcdefghijklmnopqrstuvwxyz'

def error_maker(text, coef):
    """
    Simulate typing errors in a given text based on a specified coefficient.

    :param text: The original text to introduce errors into.
    :param coef: A coefficient indicating the fraction of the text to be modified with errors.
    :return: The modified text with simulated typing errors.
    """
    num_errors = int(len(text) * coef)
    new_text = text

    for _ in range(num_errors):
        # Choose a random position in the text, avoiding leading or trailing spaces for certain operations
        pos = random.randrange(len(new_text))
        
        # Select a random operation and a random letter for replacement or insertion
        op = random.choice(ops)
        random_letter = random.choice(letters)
        
        # Split the text at the chosen position
        L, R = new_text[:pos], new_text[pos:]
        
        # Apply the operation
        new_text = op(L, R, random_letter)

    return new_text

### Evaluation

In [16]:
def calculate_similarity(text1, text2):
    """
    Calculate the similarity ratio between two texts using SequenceMatcher.
    
    :param text1: The first text for comparison.
    :param text2: The second text for comparison.
    :return: A float representing the similarity ratio, where 1.0 is identical, and 0 is completely different.
    """
    sequence_matcher = difflib.SequenceMatcher(a=text1, b=text2)
    return sequence_matcher.ratio()

In [17]:
erroneous_sentences = [error_maker(sentence, 0.1) for sentence in sentences]

In [18]:
similarity_product_custom_correction = 1.0
similarity_product_norvig_correction = 1.0

In [19]:
for erroneous_sentence, correct_sentence_text in zip(erroneous_sentences, sentences):
    # Correct the sentence using the custom correction algorithm
    corrected_custom = ''.join(correct_sentence(erroneous_sentence))
    similarity_custom = calculate_similarity(correct_sentence_text, corrected_custom)
    similarity_product_custom_correction *= similarity_custom

    # Correct the sentence using Norvig's correction algorithm
    corrected_norvig = ' '.join(correct_sentence_norvig(erroneous_sentence))
    similarity_norvig = calculate_similarity(correct_sentence_text, corrected_norvig)
    similarity_product_norvig_correction *= similarity_norvig

    # Optionally, uncomment to print sentences and their similarity scores
    print(f"Erroneous: {erroneous_sentence}\nCorrected (Custom): {corrected_custom}\nCorrected (Norvig): {corrected_norvig}")
    print(f"Similarity (Custom): {similarity_custom}\nSimilarity (Norvig): {similarity_norvig}\n")

Correcting sentence:   0%|          | 0/20 [00:00<?, ?it/s]

Correcting sentence: 100%|██████████| 20/20 [00:05<00:00,  3.35it/s]


Erroneous: olmes had sat up uopwn thez couch and i saw him motiop like a ma nhwo is icn need of air
Corrected (Custom): holmes had sat up down the couch and i saw him motion like a a who is in need of air
Corrected (Norvig): holmes had sat up down the couch and i saw him motion like a ma who is in need of air
Similarity (Custom): 0.9647058823529412
Similarity (Norvig): 0.9707602339181286



Correcting sentence: 100%|██████████| 8/8 [00:02<00:00,  2.83it/s]


Erroneous: a maid aushed across wand tbhrew open the windw
Corrected (Custom): a maid rushed across and threw open the window
Corrected (Norvig): a maid rushed across wand threw open the window
Similarity (Custom): 1.0
Similarity (Norvig): 0.989247311827957



...

Erroneous: i vcaught a glimpse of rushing fguraesatnd a moment lazer the voice ofh olmes from iwthin assuring them thav sit was a falee alarm
Corrected (Custom): i caught a glimpse of rushing fguraesatnd a moment later the voice of holmes from within assuring them that it was a false alarm
Corrected (Norvig): i caught a glimpse of rushing fguraesatnd a moment later the voice of holmes from within assuring them that sit was a false alarm
Similarity (Custom): 0.984375
Similarity (Norvig): 0.980544747081712



Correcting sentence: 100%|██████████| 35/35 [00:12<00:00,  2.79it/s]


Erroneous: sloipping htrougyh the shotuing crowd  made ym way to the cwrner of the street mnd in ten minutes aws reojiced to find my friend sam in mine nd t oget awax from thez seene of uproar
Corrected (Custom): slipping through the shouting crowd made my way to the corner of the street and in ten minutes as rejoiced to find my friend sam in mine and t get away from the scene of uproar
Corrected (Norvig): slipping through the shouting crowd made my way to the corner of the street and in ten minutes was rejoiced to find my friend sam in mine nd t get away from the seen of uproar
Similarity (Custom): 0.9803921568627451
Similarity (Norvig): 0.9719101123595506



Correcting sentence: 100%|██████████| 24/24 [00:09<00:00,  2.54it/s]


Erroneous: he waplked sbwiftlvq and in ilence for soxe few minutes until ww had turned dsown onejfo the quiet streets whih lead towards the edgemwar eroad
Corrected (Custom): he walked sbwiftlvq and in silence for some few minutes until we had turned down onejfo the quiet streets which lead towards the edgeware road
Corrected (Norvig): he walked sbwiftlvq and in silence for some few minutes until we had turned down onejfo the quiet streets which lead towards the edgeware road
Similarity (Custom): 0.9716312056737588
Similarity (Norvig): 0.9716312056737588



In [20]:
print(f"Final Similarity Product (Custom Correction): {similarity_product_custom_correction}")
print(f"Final Similarity Product (Norvig Correction): {similarity_product_norvig_correction}")

Final Similarity Product (Custom Correction): 0.8183660635543504
Final Similarity Product (Norvig Correction): 0.7895741981728733


### The final computed similarity products are as follows:

Custom Correction: 0.818

Norvig's Correction: 0.789

This data indicates that the Custom Correction method outperforms Norvig's Correction method, achieving a higher similarity product. Specifically, the Custom Correction method achieves a similarity product of approximately 0.818, whereas Norvig's Correction method achieves about 0.789. This comparison suggests that the Custom Correction method is more effective in aligning the corrected sentences closer to the target sentences when evaluated using the similarity product metric.

However, it's important to approach the comparison of these results with an understanding of the potential influences on performance. Factors such as the characteristics of the test dataset and the foundational assumptions of each algorithm could significantly impact their respective accuracies. This underscores the necessity of a nuanced evaluation when comparing the efficacy of different spelling correction implementations.

### For extra checks

In [21]:
original_sentence = "Elephants love dancing on rainbows after midnight."
erroneous_sentence = "Elaphnts lvoe dancng on ranibows afer mdinight."

corrected_sentence = ''.join(correct_sentence(erroneous_sentence))

print(f"Original: {original_sentence}")
print(f"Erroneous: {erroneous_sentence}")
print(f"Corrected: {corrected_sentence}")

similarity_score = calculate_similarity(original_sentence, corrected_sentence)
print(f"Similarity Score: {similarity_score}")


Correcting sentence: 100%|██████████| 7/7 [00:01<00:00,  3.66it/s]

Original: Elephants love dancing on rainbows after midnight.
Erroneous: Elaphnts lvoe dancng on ranibows afer mdinight.
Corrected: elephants love dancing on rainbow after midnight .
Similarity Score: 0.96
